In [2]:
# Import Dependencies
import pandas as pd

In [3]:
def get_cel_value(df, col_name):
    return df.at[0,col_name]

In [7]:
# Make a reference to the books.csv file path
entpre_acq_file = "../Resources/Census_ASE1/ASE_2016_00CSCBO01.csv"

# Import the books.csv file as a DataFrame
entpre_acq_dirty_df = pd.read_csv(entpre_acq_file, encoding="utf-8")
entpre_acq_dirty_df.head(2)

,GEO.id,GEO.id2,GEO.display-label,GEO.annotation.id,NAICS.id,NAICS.display-label,NAICS.annotation.id,ASECBO.id,ASECBO.display-label,YIBSZFI.id,YIBSZFI.display-label,ACQBUS.id,ACQBUS.display-label,YEAR.id,OWNPDEMP,OWNPDEMP_PCT,OWNPDEMP_S,OWNPDEMP_PCT_S
0,Geographic identifier code,Id2,Geographic area name,Id,2012 NAICS code,Meaning of 2012 NAICS code,Id,"Gender, ethnicity, race, and veteran status code","Meaning of Gender, ethnicity, race, and vetera...",Years in business code,Meaning of Years in business code,How the owner initially acquired the business ...,Meaning of How the owner initially acquired th...,Year,Number of owners of respondent firms with paid...,Percent of number of owners of respondent firm...,Relative standard error of number of owners of...,Standard error of percent of number of owners ...
1,0100000US,NaN,United States,NaN,00,Total for all sectors,NaN,00,All owners of respondent firms,001,All firms,CA,Founded or started,2016,3641879,69.7,0.2,0.3


In [8]:
# drop unneeded colums
entpre_acq_dirty_df.drop(['GEO.id2', 'GEO.annotation.id', 'NAICS.id', 'NAICS.annotation.id', 'ASECBO.id','YIBSZFI.id','ACQBUS.id'], axis=1, inplace=True)
entpre_acq_dirty_df.head(2)

,GEO.id,GEO.display-label,NAICS.display-label,ASECBO.display-label,YIBSZFI.display-label,ACQBUS.display-label,YEAR.id,OWNPDEMP,OWNPDEMP_PCT,OWNPDEMP_S,OWNPDEMP_PCT_S
0,Geographic identifier code,Geographic area name,Meaning of 2012 NAICS code,"Meaning of Gender, ethnicity, race, and vetera...",Meaning of Years in business code,Meaning of How the owner initially acquired th...,Year,Number of owners of respondent firms with paid...,Percent of number of owners of respondent firm...,Relative standard error of number of owners of...,Standard error of percent of number of owners ...
1,0100000US,United States,Total for all sectors,All owners of respondent firms,All firms,Founded or started,2016,3641879,69.7,0.2,0.3


In [9]:
# rename columns using first row
entpre_acq_dirty_df.columns = [get_cel_value(entpre_acq_dirty_df, x) for x in entpre_acq_dirty_df.columns]
# remove first row
entpre_acq_dirty_df.drop(entpre_acq_dirty_df.index[0], inplace=True)
# remove columns with empty cells
entpre_acq_dirty_df.dropna(axis=1, how='all', inplace=True)
entpre_acq_dirty_df.head(5)

,Geographic identifier code,Geographic area name,Meaning of 2012 NAICS code,"Meaning of Gender, ethnicity, race, and veteran status code",Meaning of Years in business code,Meaning of How the owner initially acquired the business code,Year,Number of owners of respondent firms with paid employees,Percent of number of owners of respondent firms with paid employees (%),Relative standard error of number of owners of respondent firms with paid employees (%),Standard error of percent of number of owners of respondent firms with paid employees (%)
1,0100000US,United States,Total for all sectors,All owners of respondent firms,All firms,Founded or started,2016,3641879,69.7,0.2,0.3
2,0100000US,United States,Total for all sectors,All owners of respondent firms,All firms,Purchased,2016,1114797,21.3,0.6,0.2
3,0100000US,United States,Total for all sectors,All owners of respondent firms,All firms,Inherited,2016,215576,4.1,1.2,0.1
4,0100000US,United States,Total for all sectors,All owners of respondent firms,All firms,Transfer of ownership or gift,2016,380986,7.3,1.4,0.1
5,0100000US,United States,Total for all sectors,All owners of respondent firms,All firms,Total reporting,2016,5223412,100.0,0.1,0.0


In [10]:
for col_name in entpre_acq_dirty_df.columns:
    print(col_name)

Geographic identifier code
Geographic area name
Meaning of 2012 NAICS code
Meaning of Gender, ethnicity, race, and veteran status code
Meaning of Years in business code
Meaning of How the owner initially acquired the business code
Year
Number of owners of respondent firms with paid employees 
Percent of number of owners of respondent firms with paid employees (%) 
Relative standard error of number of owners of respondent firms with paid employees (%) 
Standard error of percent of number of owners of respondent firms with paid employees (%) 
